**Задание. Написать приложение и функцию, которые собирают основные новости с сайта на выбор dzen.ru, lenta.ru, mail.ru . Для парсинга использовать XPath
Структура данных должна содержать:** 

*** название источника** 

*** наименование новости** 

*** ссылку на новость** 

*** дата публикации**


In [ ]:
from pprint import pprint
from lxml import html
import requests
import pandas as pd

In [ ]:
USER_AGENT = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36'
URL_LENTA_NEWS = 'https://lenta.ru/'
URL_DZEN_NEWS = 'https://dzen.ru/news/'

In [ ]:
headers = {
    'User-Agent': USER_AGENT,
}

params = {
    'sso_failed': '',
}

In [ ]:
def get_content_dom_from_html_text(url, headers=None, params=None):
    response = requests.get(url, headers=headers, params=params)
    content_dom = html.fromstring(response.text)

    print(response.url)
    
    return content_dom

In [ ]:
def parser_dzen_news(content_dom):
    news_container = content_dom.xpath("//div[contains(@class,'mg-card ')]")
    dzen_news = []
    for new_container in news_container:
        new_source = new_container.xpath(".//a[@class='mg-card__source-link']/text()")[0]
        new_text = new_container.xpath(".//h2/a/text()")[0].replace('\xa0', ' ')
        new_link = new_container.xpath(".//h2/a/@href")[0]
        new_date = new_container.xpath(".//span[@class='mg-card-source__time']/text()")[0]
        news_dict = {
            'new_source': new_source,
            'new_text': new_text,
            'new_link': new_link,
            'new_date': new_date,
        }
        dzen_news.append(news_dict) 

    return dzen_news

In [ ]:
def parser_lenta_news(content_dom):
    news_container = content_dom.xpath("//a[contains(@class, 'card-')]")
    lenta_news = []
    for new_container in news_container:
        new_source = new_container.xpath(".//*[name()='svg']/*[name()='use']/attribute::*")
        if len(new_source) == 1:
            new_source = new_source[0].split('ui-label_')
            if len(new_source) == 2:
                new_source = new_source[1]
            else:
                new_source = 'Lenta.ru'
        else:
            new_source = 'Lenta.ru' 
        new_text = new_container.xpath(".//span[contains(@class, 'card-')]/text()")
        if len(new_text) == 1:
            new_text = new_text[0]
        else:
            new_text = None  
        new_link = URL_LENTA_NEWS + new_container.xpath("./@href")[0]
        
        new_date = new_container.xpath(".//time/text()")
        if len(new_date) == 1:
            new_date = new_date[0]
        else:
            new_date = None
        news_dict = {
            'new_source': new_source,
            'new_text': new_text,
            'new_link': new_link,
            'new_date': new_date,
        }
        lenta_news.append(news_dict)
        
    return lenta_news

In [ ]:
def get_news(content_dom, parser_name='dzen'):
    result = None
    
    if parser_name == 'dzen':
      result = parser_dzen_news(content_dom)
    elif parser_name == 'yandex':
      result = parser_yandex_news(content_dom)
    elif parser_name == 'lenta':
      result = parser_lenta_news(content_dom)
    
    return result

In [ ]:
dom = get_content_dom_from_html_text(URL_DZEN_NEWS, headers=headers, params=params)

https://dzen.ru/news?issue_tld=ru&sso_failed=


In [ ]:
dzen_news = get_news(dom, 'dzen')
dzen_news

[{'new_source': 'Lenta.ru',
  'new_text': 'Захарова призвала Нуланд заняться темой неопознанных объектов над США, а не Украиной',
  'new_link': 'https://dzen.ru/news/story/Zakharova_prizvala_Nuland_zanyatsya_temoj_neopoznannykh_obektov_nadSSHA_a_ne_Ukrainoj--fef215a6be2412e1689d20c2f7f4a7a7?lang=ru&rubric=index&fan=1&stid=URiYYhHYQAUo1YvLE9z2&t=1676628927&tt=true&persistent_id=2016160795&story=38f5f68e-29df-514b-8870-f1b3a52fa9ce&issue_tld=ru',
  'new_date': '03:04'},
 {'new_source': 'RT на русском',
  'new_text': 'Песков заявил, что по вопросу разового взноса бизнеса в бюджет идёт обсуждение',
  'new_link': 'https://dzen.ru/news/story/Peskov_zayavil_chto_povoprosu_razovogo_vznosa_biznesa_vbyudzhet_idyot_obsuzhdenie--c1465cab05565f2ffe01d8ea38a47c8a?lang=ru&rubric=index&fan=1&stid=t2ebtKbWx6RZeRYQuUDG&t=1676628927&tt=true&persistent_id=2016161327&story=aaa5e200-01b5-569c-9614-78dfa77d1737&issue_tld=ru',
  'new_date': '03:05'},
 {'new_source': 'ТАСС',
  'new_text': 'ВЦИОМ: уровень довер

In [ ]:
dom = get_content_dom_from_html_text(URL_LENTA_NEWS, headers=headers, params=params)

https://lenta.ru/?sso_failed=


In [ ]:
lenta_news = get_news(dom, 'lenta')
lenta_news

[{'new_source': 'Lenta.ru',
  'new_text': None,
  'new_link': 'https://lenta.ru//news/2023/02/17/zakhaarova_nuland/',
  'new_date': '12:23'},
 {'new_source': 'Lenta.ru',
  'new_text': 'В МИДе назвали антинаучным признание Репина и Айвазовского украинцами',
  'new_link': 'https://lenta.ru//news/2023/02/17/repinartist/',
  'new_date': '13:23'},
 {'new_source': 'Lenta.ru',
  'new_text': 'В Сумах прогремел взрыв',
  'new_link': 'https://lenta.ru//news/2023/02/17/vzrivsum/',
  'new_date': '13:23'},
 {'new_source': 'Lenta.ru',
  'new_text': 'Захарова потребовала от США предоставить факты по делу «Северных потоков»',
  'new_link': 'https://lenta.ru//news/2023/02/17/weneedfacts/',
  'new_date': '13:22'},
 {'new_source': 'Lenta.ru',
  'new_text': 'Вероятность вступления США и Китая в войну из-за Тайваня оценили',
  'new_link': 'https://lenta.ru//news/2023/02/17/usa_china/',
  'new_date': '13:21'},
 {'new_source': 'Lenta.ru',
  'new_text': 'Военный ВСУ пожаловался на работу танка вдвое старше ег